<a href="https://colab.research.google.com/github/blue0620/NextDLAnalyzer/blob/main/disedigietextdownloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 次世代デジタルライブラリーからのテキストデータのダウンロードインターフェース


In [ ]:
#@title 検索したいキーワードを入れてください。絞り込まない項目は空欄にしてください
Keyword="鴎外"#@param {type:"string", placeholder:"検索キーワードを入力（例：鴎外）"}
#@markdown ###以下は絞込条件です
Title=""#@param {type:"string",placeholder:"タイトルで絞り込む"}
Author=""#@param {type:"string",placeholder:"著者で絞り込む"}
Yearstart=""#@param {type:"string",placeholder:"出版年始点（例：1900）"}
Yearend=""#@param {type:"string",placeholder:"出版年終点（例：1945）"}
NDC = ""#@param {type:"string",placeholder:"NDCで絞り込む（例：大分類9の場合9、中分類91の場合91）"}

In [ ]:
import requests
import time

def jpsearch_scroll(database: str = "dignl", delay: float = 0.5):
    endpoint = "https://jpsearch.go.jp/api/item/scroll/jps-cross"

    # 初回リクエスト
    params = {"f-db": database, "f-rights": "pdm"}
    if Keyword:
        params["keyword"]=Keyword
    if Title:
        params["q-common.title"]=Title
    if Author:
        params["q-common.author"]=Author
    if Yearstart and Yearend:
        params["r-tempo"]=Yearstart+","+Yearend
    elif Yearstart:
        params["r-tempo"]=Yearstart+","
    elif Yearend:
        params["r-tempo"]=","+Yearend
    response = requests.get(endpoint, params=params)
    response.raise_for_status()
    data = response.json()

    # 最初の結果を処理
    #print(data)
    resitems=[]
    for item in data.get("list", []):
        resitems.append(item)

    scroll_id = data.get("scrollId")

    # scrollを使って次のページを繰り返し取得
    while scroll_id:
        time.sleep(delay)
        params = {"scrollId": scroll_id}
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        data = response.json()

        items = data.get("list", [])
        if not items:
            break

        for item in items:
            resitems.append(item)
        scroll_id = data.get("scrollId")
    return resitems
rawreslist=jpsearch_scroll()
reslist=[]

if NDC:
    for tmp in rawreslist:
        try:
            ndcval=tmp["dignl-rdf:RDF"]["dcndl:BibResource"]["dc:subject_rdf:datatype_http://ndl_go_jp/dcndl/terms/NDC-s"][0]
        except:
            continue
        if ndcval.startswith(NDC):
            reslist.append(tmp)
else:
    reslist=rawreslist
print(len(reslist))

In [ ]:
import requests
import zipfile
import io
import os
from tqdm import tqdm
import shutil

extract_to="ocrtext"
# 出力ディレクトリを初期化
if os.path.exists(extract_to):
    shutil.rmtree(extract_to)
os.makedirs(extract_to)
print(f"ディレクトリ '{extract_to}' を作成しました。")
count_ok=0
for item in tqdm(reslist):
    PID=item["id"].split("-")[1]
    title=item["common"]["title"]
    try:
        url=f"https://lab.ndl.go.jp/dl/api/book/layouttext/{PID}"
        # URLからデータを取得
        response = requests.get(url)
        # HTTPエラーがあれば例外を発生させる
        response.raise_for_status()
        print(f"{PID}_{title}  のダウンロードが完了しました。")
        # ダウンロードしたコンテンツをメモリ上でZIPファイルとして開く
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            #print(f"ファイルを '{extract_to}' ディレクトリに展開しています...")
            # 全てのファイルを指定されたディレクトリに展開
            z.extractall(extract_to)
        #print("ファイルの展開が完了しました。")
        #print(f"展開先: {os.path.abspath(extract_to)}")
        count_ok+=1
    except requests.exceptions.RequestException as e:
        #print(f"ダウンロード中にエラーが発生しました: {e}")
        #print(f"ダウンロード未対応の書誌のためスキップします")
        pass
    except zipfile.BadZipFile:
        print("ダウンロードしたファイルは有効なZIPファイルではありません。")
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")

ダウンロードしたZIPファイルを展開したディレクトリから、すべてのXMLファイル名のリストを取得します。


In [ ]:
import os
import glob
import shutil
import xml.etree.ElementTree as ET
directory_path = 'ocrtext'
outputdirectory_path='outputtextdata'
if os.path.exists(outputdirectory_path):
    shutil.rmtree(outputdirectory_path)
os.makedirs(outputdirectory_path)
for item in tqdm(reslist):
    PID=item["id"].split("-")[1]
    title=item["common"]["title"]
    #all_files = os.listdir(directory_path)
    xml_files = glob.glob(os.path.join(directory_path,PID+"*.xml"))
    if len(xml_files)==0:
        continue
    xml_files.sort()
    sorted_xml_files = xml_files
    extracted_texts=[]
    for file_path in sorted_xml_files:
        try:
            tree = ET.parse(file_path)
            root = tree.getroot()
            for text_element in root.findall('.//LINE'):
                if text_element.get("STRING"):
                    extracted_texts.append(text_element.get("STRING").strip())
        except ET.ParseError as e:
            print(f"Error parsing {file_path}: {e}")
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")
    combined_text = "\n".join(extracted_texts)
    filename=PID+"_"+title.replace(" ","")+".txt"
    with open(os.path.join(outputdirectory_path,filename),"w",encoding="utf-8") as wf:
        wf.write(combined_text)
shutil.make_archive('outputtextdata', format='zip', root_dir=outputdirectory_path)

In [ ]:
import ipywidgets as widgets
from google.colab import files
from IPython.display import display

# 3. ダウンロードボタンを作成
# description引数でボタンに表示するテキストを設定
txtbutton = widgets.Button(description="テキストDL")

# 4. ボタンがクリックされたときに実行される関数を定義
def on_button_clicked(b):
  # files.download() を使ってファイルをダウンロード
  files.download('outputtextdata.zip')

# 5. ボタンのクリックイベントに関数を登録
txtbutton.on_click(on_button_clicked)

# 6. ボタンを画面に表示
display(txtbutton)


## 以降は取得したテキストデータの分析（新仮名旧仮名判定）

In [ ]:
!wget https://github.com/blue0620/NextDLAnalyzer/raw/refs/heads/main/trained_model.pkl -O trained_model.pkl
!wget https://github.com/blue0620/NextDLAnalyzer/raw/refs/heads/main/vectorizer.pkl -O vectorizer.pkl

In [ ]:
import pickle
import lightgbm
import re
from sklearn.feature_extraction.text import CountVectorizer
pattern = r'[A-Za-z0-9,]'
vectorizer= pickle.load(open('vectorizer.pkl', 'rb'))
model = pickle.load(open('trained_model.pkl', 'rb'))

with open("oldnewanalyze.csv","w") as wf:
    wf.write("判定結果,ファイル名\n")
    for txtpath in glob.glob("outputtextdata/*.txt"):
        txtname=os.path.basename(txtpath).split(".")[0]
        with open(txtpath,"r",encoding="utf-8") as rf:
            combined_text=rf.read()
            combined_text=combined_text.replace("\n","").replace(" ","")
            combined_text=re.sub(pattern, '', combined_text)
            X_te = vectorizer.transform([combined_text])
            y_pred_proba = model.predict(X_te.astype('float32'), num_iteration=model.best_iteration)
            if y_pred_proba[0]>0.5:
                wf.write(f"新かな,{txtname}\n")
                print(combined_text[:1000],y_pred_proba)
            else:
                wf.write(f"旧かな,{txtname}\n")

In [ ]:
import ipywidgets as widgets
from google.colab import files
from IPython.display import display

analyzebutton = widgets.Button(description="判定結果DL")
def on_button_clicked2(b):
  # files.download() を使ってファイルをダウンロード
  files.download('oldnewanalyze.csv')
analyzebutton.on_click(on_button_clicked2)
display(analyzebutton)